# Dense Dataset Update Status

**✅ This project now operates exclusively on dense, optimized datasets:**

## Dense Dataset Files Created:
- **train_dense.csv**: 948K interactions (was 6.3M) - 95% reduction
- **valid_dense.csv**: 30K interactions (was 1.1M) - 97% reduction  
- **test_dense.csv**: 29K interactions (was 1.0M) - 97% reduction
- **metadata_dense.csv**: 16.6K products (was 149K) - 89% reduction

## Optimization Results:
- **Users**: 67K active users (≥10 interactions each)
- **Products**: 16.6K popular products (≥15 users each)
- **Interactions**: 1M total (12.4% of original)
- **Average user activity**: 15 interactions (vs 5 originally)
- **File sizes**: 95% smaller overall
- **Matrix density**: Significantly improved for collaborative filtering

## Schema Standardization:
- **Interactions**: `user_id`, `product_id`, `rating`, `timestamp`
- **Metadata**: `product_id`, `title`, `main_category`, `price`, `average_rating`, etc.

All downstream systems (database, models, API, frontend) have been updated to use these optimized datasets exclusively.

# Data Cleaning and Preprocessing

Data cleaning and final dataset preparation for recommendation system.

## Objectives
1. Clean interaction datasets
2. Validate metadata
3. Ensure product-metadata alignment
4. Create final model-ready datasets

In [11]:
# 1. Import libraries
import pandas as pd
import numpy as np
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')

print("Libraries imported")

Libraries imported


## 2. Load Datasets

In [1]:
# 2.1 Load original datasets
data_path = Path("../01_metadata_processing")

print("Loading datasets...")

# Load interaction datasets
datasets = {}
for name in ['train', 'valid', 'test']:
    file_path = data_path / f"{name}_cleaned.csv"
    print(f"Loading {name}_cleaned.csv...")
    df = pd.read_csv(file_path)
    datasets[name] = df
    print(f"  {name}: {len(df):,} rows")

# Load metadata
print("Loading metadata_filtered.csv...")
metadata = pd.read_csv(data_path / "metadata_filtered.csv")
print(f"  metadata: {len(metadata):,} products")

print("\nDatasets loaded")

NameError: name 'Path' is not defined

## 3. Clean Interactions

In [13]:
# 3.1 Clean interaction datasets
print("CLEANING INTERACTIONS")
print("=" * 25)

cleaned_data = {}

for name, df in datasets.items():
    print(f"\nCleaning {name}...")
    
    clean_df = df.copy()
    original_len = len(clean_df)
    
    # Fix data types
    clean_df['user_id'] = clean_df['user_id'].astype('string')
    clean_df['parent_asin'] = clean_df['parent_asin'].astype('string')
    clean_df['rating'] = pd.to_numeric(clean_df['rating'], errors='coerce')
    clean_df['timestamp'] = pd.to_datetime(clean_df['timestamp'])
    clean_df['history'] = clean_df['history'].fillna('').astype('string')
    
    # Remove missing critical data
    clean_df = clean_df.dropna(subset=['user_id', 'parent_asin', 'rating'])
    print(f"  After removing missing: {len(clean_df):,}")
    
    # Filter valid ratings (1-5)
    clean_df = clean_df[(clean_df['rating'] >= 1) & (clean_df['rating'] <= 5)]
    print(f"  After rating filter: {len(clean_df):,}")
    
    # Remove duplicates
    clean_df = clean_df.drop_duplicates(subset=['user_id', 'parent_asin', 'timestamp'])
    print(f"  After deduplication: {len(clean_df):,}")
    
    cleaned_data[name] = clean_df
    rows_removed = original_len - len(clean_df)
    print(f"  Removed: {rows_removed:,} rows")

print("\nInteraction cleaning complete")

CLEANING INTERACTIONS

Cleaning train...
  After removing missing: 12,191,484
  After removing missing: 12,191,484
  After rating filter: 12,191,484
  After rating filter: 12,191,484
  After deduplication: 12,191,484
  Removed: 0 rows

Cleaning valid...
  After deduplication: 12,191,484
  Removed: 0 rows

Cleaning valid...
  After removing missing: 1,641,026
  After rating filter: 1,641,026
  After removing missing: 1,641,026
  After rating filter: 1,641,026
  After deduplication: 1,641,026
  Removed: 0 rows

Cleaning test...
  After deduplication: 1,641,026
  Removed: 0 rows

Cleaning test...
  After removing missing: 1,641,026
  After rating filter: 1,641,026
  After removing missing: 1,641,026
  After rating filter: 1,641,026
  After deduplication: 1,641,026
  Removed: 0 rows

Interaction cleaning complete
  After deduplication: 1,641,026
  Removed: 0 rows

Interaction cleaning complete


## 4. Clean Metadata

In [14]:
# 4.1 Clean metadata
print("CLEANING METADATA")
print("=" * 20)

clean_meta = metadata.copy()
original_len = len(clean_meta)

print(f"Original: {original_len:,} products")

# Fix data types
clean_meta['parent_asin'] = clean_meta['parent_asin'].astype('string')
clean_meta['title'] = clean_meta['title'].fillna('').astype('string')
clean_meta['main_category'] = clean_meta['main_category'].fillna('').astype('string')
clean_meta['store'] = clean_meta['store'].fillna('').astype('string')
clean_meta['categories'] = clean_meta['categories'].fillna('').astype('string')

# Clean numeric fields
clean_meta['average_rating'] = pd.to_numeric(clean_meta['average_rating'], errors='coerce')
clean_meta['rating_number'] = pd.to_numeric(clean_meta['rating_number'], errors='coerce')
clean_meta['price'] = pd.to_numeric(clean_meta['price'], errors='coerce')

# Remove duplicates
clean_meta = clean_meta.drop_duplicates(subset=['parent_asin'])
print(f"After deduplication: {len(clean_meta):,}")

# Quality stats
print(f"\nQuality:")
print(f"  With prices: {clean_meta['price'].notna().sum():,} ({clean_meta['price'].notna().sum()/len(clean_meta)*100:.1f}%)")
print(f"  With ratings: {clean_meta['average_rating'].notna().sum():,} ({clean_meta['average_rating'].notna().sum()/len(clean_meta)*100:.1f}%)")

print("\nMetadata cleaning complete")

CLEANING METADATA
Original: 149,636 products
After deduplication: 149,636

Quality:
  With prices: 78,138 (52.2%)
  With ratings: 149,636 (100.0%)

Metadata cleaning complete
Original: 149,636 products
After deduplication: 149,636

Quality:
  With prices: 78,138 (52.2%)
  With ratings: 149,636 (100.0%)

Metadata cleaning complete


## 5. Coverage Validation

In [15]:
# 5.1 Validate product coverage
print("COVERAGE VALIDATION")
print("=" * 25)

# Get products from interactions
interaction_products = set()
for name, df in cleaned_data.items():
    interaction_products.update(df['parent_asin'].unique())
    print(f"{name}: {df['parent_asin'].nunique():,} products")

print(f"\nTotal interaction products: {len(interaction_products):,}")

# Get products from metadata
meta_products = set(clean_meta['parent_asin'].unique())
print(f"Metadata products: {len(meta_products):,}")

# Calculate overlap
overlap = interaction_products.intersection(meta_products)
missing_meta = interaction_products - meta_products
extra_meta = meta_products - interaction_products

coverage_pct = (len(overlap) / len(interaction_products)) * 100

print(f"\nCoverage:")
print(f"  With metadata: {len(overlap):,}")
print(f"  Coverage: {coverage_pct:.1f}%")
print(f"  Missing metadata: {len(missing_meta):,}")
print(f"  Extra metadata: {len(extra_meta):,}")

if coverage_pct >= 50:
    print(f"\nGood coverage ({coverage_pct:.1f}%)")
else:
    print(f"\nWarning: Low coverage ({coverage_pct:.1f}%)")

COVERAGE VALIDATION
train: 367,052 products
train: 367,052 products
valid: 251,202 products
valid: 251,202 products
test: 231,672 products

Total interaction products: 368,228
Metadata products: 149,636

Coverage:
  With metadata: 149,636
  Coverage: 40.6%
  Missing metadata: 218,592
  Extra metadata: 0

test: 231,672 products

Total interaction products: 368,228
Metadata products: 149,636

Coverage:
  With metadata: 149,636
  Coverage: 40.6%
  Missing metadata: 218,592
  Extra metadata: 0



## 6. Create Final Datasets

In [16]:
# 6.1 Create final aligned datasets
print("CREATING FINAL DATASETS")
print("=" * 30)

# Get products with metadata
products_with_meta = set(clean_meta['parent_asin'].unique())

# Filter interactions to products with metadata
final_data = {}
for name, df in cleaned_data.items():
    filtered_df = df[df['parent_asin'].isin(products_with_meta)].copy()
    final_data[name] = filtered_df
    
    removed = len(df) - len(filtered_df)
    print(f"{name}: {len(df):,} -> {len(filtered_df):,} ({removed:,} removed)")
    print(f"  Users: {filtered_df['user_id'].nunique():,}")
    print(f"  Products: {filtered_df['parent_asin'].nunique():,}")

# Filter metadata to products in interactions
all_final_products = set()
for df in final_data.values():
    all_final_products.update(df['parent_asin'].unique())

final_meta = clean_meta[clean_meta['parent_asin'].isin(all_final_products)].copy()
print(f"\nMetadata: {len(clean_meta):,} -> {len(final_meta):,}")

# Summary
total_interactions = sum(len(df) for df in final_data.values())
total_users = len(set().union(*[df['user_id'].unique() for df in final_data.values()]))
total_products = len(all_final_products)

print(f"\nFinal Summary:")
print(f"  Interactions: {total_interactions:,}")
print(f"  Users: {total_users:,}")
print(f"  Products: {total_products:,}")
print(f"  Metadata: {len(final_meta):,}")
print(f"  Coverage: 100.0%")

CREATING FINAL DATASETS
train: 12,191,484 -> 6,267,269 (5,924,215 removed)
train: 12,191,484 -> 6,267,269 (5,924,215 removed)
  Users: 1,560,254
  Users: 1,560,254
  Products: 149,111
  Products: 149,111
valid: 1,641,026 -> 910,433 (730,593 removed)
  Users: 910,433
  Products: 107,049
valid: 1,641,026 -> 910,433 (730,593 removed)
  Users: 910,433
  Products: 107,049
test: 1,641,026 -> 944,716 (696,310 removed)
  Users: 944,716
  Products: 100,544
test: 1,641,026 -> 944,716 (696,310 removed)
  Users: 944,716
  Products: 100,544

Metadata: 149,636 -> 149,636

Metadata: 149,636 -> 149,636

Final Summary:
  Interactions: 8,122,418
  Users: 1,624,138
  Products: 149,636
  Metadata: 149,636
  Coverage: 100.0%

Final Summary:
  Interactions: 8,122,418
  Users: 1,624,138
  Products: 149,636
  Metadata: 149,636
  Coverage: 100.0%


## 7. Save Final Files

In [17]:
# 7.1 Save final datasets
print("SAVING FILES")
print("=" * 15)

output_path = Path(".")

# Save interaction files
for name, df in final_data.items():
    filename = output_path / f"{name}_final.csv"
    df.to_csv(filename, index=False)
    size_mb = filename.stat().st_size / 1024**2
    print(f"{name}_final.csv: {len(df):,} rows, {size_mb:.1f} MB")

# Save metadata
meta_filename = output_path / "metadata_final.csv"
final_meta.to_csv(meta_filename, index=False)
meta_size = meta_filename.stat().st_size / 1024**2
print(f"metadata_final.csv: {len(final_meta):,} products, {meta_size:.1f} MB")

print("\nAll files saved")

SAVING FILES
train_final.csv: 6,267,269 rows, 970.0 MB
train_final.csv: 6,267,269 rows, 970.0 MB
valid_final.csv: 910,433 rows, 130.6 MB
valid_final.csv: 910,433 rows, 130.6 MB
test_final.csv: 944,716 rows, 144.8 MB
test_final.csv: 944,716 rows, 144.8 MB
metadata_final.csv: 149,636 products, 36.0 MB

All files saved
metadata_final.csv: 149,636 products, 36.0 MB

All files saved


## 8. Validation Report

In [18]:
# 8.1 Validation report
print("VALIDATION REPORT")
print("=" * 20)

results = []

# Check data types
print("\n1. Data Types")
for name, df in final_data.items():
    types_ok = (
        df['user_id'].dtype == 'string' and
        df['parent_asin'].dtype == 'string' and
        np.issubdtype(df['rating'].dtype, np.floating) and
        np.issubdtype(df['timestamp'].dtype, np.datetime64)
    )
    status = "Pass" if types_ok else "Fail"
    print(f"  {name}: {status}")
    results.append(types_ok)

# Check missing values
print("\n2. Missing Values")
for name, df in final_data.items():
    missing_critical = df[['user_id', 'parent_asin', 'rating']].isnull().sum().sum()
    no_missing = missing_critical == 0
    status = "Pass" if no_missing else "Fail"
    print(f"  {name}: {status}")
    results.append(no_missing)

# Check rating validity
print("\n3. Rating Validity")
for name, df in final_data.items():
    valid_ratings = ((df['rating'] >= 1) & (df['rating'] <= 5)).all()
    status = "Pass" if valid_ratings else "Fail"
    print(f"  {name}: {status}")
    results.append(valid_ratings)

# Check alignment
print("\n4. Product Alignment")
all_products = set().union(*[df['parent_asin'].unique() for df in final_data.values()])
meta_products = set(final_meta['parent_asin'].unique())
perfect_alignment = all_products <= meta_products
status = "Pass" if perfect_alignment else "Fail"
print(f"  Coverage: {status}")
results.append(perfect_alignment)

# Check duplicates
print("\n5. Duplicates")
for name, df in final_data.items():
    no_dups = not df.duplicated(['user_id', 'parent_asin', 'timestamp']).any()
    status = "Pass" if no_dups else "Fail"
    print(f"  {name}: {status}")
    results.append(no_dups)

# Overall status
all_passed = all(results)
print("\n" + "=" * 25)
if all_passed:
    print("ALL CHECKS PASSED")
    print("DATA READY FOR MODELING")
else:
    print("SOME CHECKS FAILED")
    print("REVIEW DATA BEFORE MODELING")
print("=" * 25)

VALIDATION REPORT

1. Data Types
  train: Pass
  valid: Pass
  test: Pass

2. Missing Values
  train: Pass
  valid: Pass
  train: Pass
  valid: Pass
  test: Pass

3. Rating Validity
  train: Pass
  valid: Pass
  test: Pass

4. Product Alignment
  test: Pass

3. Rating Validity
  train: Pass
  valid: Pass
  test: Pass

4. Product Alignment
  Coverage: Pass

5. Duplicates
  Coverage: Pass

5. Duplicates
  train: Pass
  train: Pass
  valid: Pass
  valid: Pass
  test: Pass

ALL CHECKS PASSED
DATA READY FOR MODELING
  test: Pass

ALL CHECKS PASSED
DATA READY FOR MODELING


## 9. Final Statistics

In [21]:
# 9.1 Final statistics
print("FINAL STATISTICS")
print("=" * 20)

# Overall stats
total_interactions = sum(len(df) for df in final_data.values())
total_users = len(set().union(*[df['user_id'].unique() for df in final_data.values()]))
total_products = len(set().union(*[df['parent_asin'].unique() for df in final_data.values()]))

print(f"\nOverall:")
print(f"Interactions: {total_interactions:,}")
print(f"Users: {total_users:,}")
print(f"Products: {total_products:,}")
print(f"Metadata: {len(final_meta):,}")
print(f"Sparsity: {(1 - total_interactions/(total_users * total_products)) * 100:.4f}%")

# Per-dataset stats
print(f"\nBy Dataset:")
for name, df in final_data.items():
    avg_rating = df['rating'].mean()
    date_span = (df['timestamp'].max() - df['timestamp'].min()).days
    
    print(f"\n{name.upper()}:")
    print(f"Interactions: {len(df):,}")
    print(f"Users: {df['user_id'].nunique():,}")
    print(f"Products: {df['parent_asin'].nunique():,}")
    print(f"Avg rating: {avg_rating:.2f}")
    print(f"Date span: {date_span:,} days")

# Metadata stats
print(f"\nMETADATA:")
print(f"Products: {len(final_meta):,}")
print(f"With prices: {final_meta['price'].notna().sum():,} ({final_meta['price'].notna().mean()*100:.1f}%)")
print(f"With ratings: {final_meta['average_rating'].notna().sum():,} ({final_meta['average_rating'].notna().mean()*100:.1f}%)")
print(f"Avg price: ${final_meta['price'].mean():.2f}")
print(f"Avg rating: {final_meta['average_rating'].mean():.2f}")
print(f"Categories: {final_meta['main_category'].nunique():,}")

FINAL STATISTICS

Overall:
Interactions: 8,122,418
Users: 1,624,138
Products: 149,636
Metadata: 149,636
Sparsity: 99.9967%

By Dataset:

TRAIN:
Interactions: 6,267,269

Overall:
Interactions: 8,122,418
Users: 1,624,138
Products: 149,636
Metadata: 149,636
Sparsity: 99.9967%

By Dataset:

TRAIN:
Interactions: 6,267,269
Users: 1,560,254
Users: 1,560,254
Products: 149,111
Avg rating: 4.31
Date span: 8,822 days

VALID:
Interactions: 910,433
Users: 910,433
Products: 149,111
Avg rating: 4.31
Date span: 8,822 days

VALID:
Interactions: 910,433
Users: 910,433
Products: 107,049
Avg rating: 4.22
Date span: 8,259 days

TEST:
Interactions: 944,716
Users: 944,716
Products: 107,049
Avg rating: 4.22
Date span: 8,259 days

TEST:
Interactions: 944,716
Users: 944,716
Products: 100,544
Avg rating: 4.13
Date span: 8,107 days

METADATA:
Products: 149,636
With prices: 78,138 (52.2%)
With ratings: 149,636 (100.0%)
Avg price: $82.81
Avg rating: 4.13
Categories: 37
Products: 100,544
Avg rating: 4.13
Date span: 